In [1]:
from ydata_profiling import ProfileReport
from hyperopt import rand, tpe
from hyperopt import fmin, tpe, hp, Trials
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [97]:
# Импорты из стандартной библиотеки
from pathlib import Path

# Установленные пакеты
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder as ohe
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [171]:
from ydata_profiling import ProfileReport

# Импорт функции для кэширования зип-файла
from ydata_profiling.utils.cache import cache_zipped_file

# Чтение датасета UCI Bank Dataset
file_name = cache_zipped_file(
        "bank-full.csv",
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip",
)
df = pd.read_csv(file_name, sep=";")

In [172]:
df['y'] = df['y'].apply(lambda x: 1 if 'yes' else 0)

print(df.shape, 'start')
encoder = ohe(sparse_output=False)
df_encoded = encoder.fit_transform(df[['job']])
df_encoded = pd.DataFrame.from_records(df_encoded, columns=encoder.get_feature_names_out())
print(df_encoded.shape, 'encoded shape')
print(df.shape, 'after encoding')
df['marital_ecdd'] = df['marital'].apply(lambda x: 0 if x == 'divorced' else 1 if x == 'single' else 2)

label_encoder = LabelEncoder()    
df['education_lbl'] = label_encoder.fit_transform(df['education'])
print(df.shape, 'after labelling 1')
label_encoder = LabelEncoder()    
df['month'] = label_encoder.fit_transform(df['month'])
print(df.shape, 'after labelling 2')

df['y'] = df['y'].apply(lambda x: 1 if 'yes' else 0)

cols = df.select_dtypes('object').columns.to_list()
df = df.drop(cols, axis=1)
print(df.shape, 'after dropping')
for col in df_encoded.columns:
    df[col] = df_encoded[col] 
print(df.shape, 'after merging')


In [173]:
from sklearn.preprocessing import FunctionTransformer

def preprocess(df): 
    print(df.shape, 'start')
    encoder = ohe(sparse_output=False)
    df_encoded = encoder.fit_transform(df[['job']])
    df_encoded = pd.DataFrame.from_records(df_encoded, columns=encoder.get_feature_names_out())
    print(df_encoded.shape, 'encoded shape')
    print(df.shape, 'after encoding')
    df['marital_ecdd'] = df['marital'].apply(lambda x: 0 if x == 'divorced' else 1 if x == 'single' else 2)

    label_encoder = LabelEncoder()    
    df['education_lbl'] = label_encoder.fit_transform(df['education'])
    print(df.shape, 'after labelling 1')
    label_encoder = LabelEncoder()    
    df['month'] = label_encoder.fit_transform(df['month'])
    print(df.shape, 'after labelling 2')
    
    # df['y'] = df['y'].apply(lambda x: 1 if 'yes' else 0)

    cols = df.select_dtypes('object').columns.to_list()
    df = df.drop(cols, axis=1)
    print(df.shape, 'after dropping')
    # df = df.merge(df_encoded, left_index=True, right_index=True)
    for col in df_encoded.columns:
        df[col] = df_encoded[col] 

    print(df.shape, 'after merging')
    
    return df

In [174]:
preprocessor = Pipeline(steps=[('preprocess', FunctionTransformer(preprocess))])

rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                         ('estimator', RandomForestClassifier())
                         ])

In [175]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('y', axis=1) , df['y'], test_size=0.3)

In [176]:
y_train.shape

(31647,)

In [177]:
rf_pipe.fit(X_train, y_train)

(31647, 16) start
(31647, 12) encoded shape
(31647, 16) after encoding
(31647, 18) after labelling 1
(31647, 18) after labelling 2
(31647, 10) after dropping
(31647, 22) after merging


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocess',
                                  FunctionTransformer(func=<function preprocess at 0x00000239ADFF0AE0>))])),
                ('estimator', RandomForestClassifier())])

In [179]:

baseline_preds = rf_pipe.predict(X_train)
baseline_preds

(31647, 18) start
(31647, 12) encoded shape
(31647, 18) after encoding
(31647, 18) after labelling 1
(31647, 18) after labelling 2
(31647, 10) after dropping
(31647, 22) after merging


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)